In [1]:
import numpy as np
import pandas as pd
import tailor

In [2]:
df = tailor.load_data()

In [3]:
df.head()

,article_id,transaction_date,markdown_start_date,markdown_end_date,original_price,sells_price,discount,markdown,article_count,season,brand,color,stock_total,avq,Abteilung,WHG,WUG,time_on_sale,revenue
0,900001,2014-03-27,2013-10-18,2014-05-31,59.95,53.95,6.0,0.0,2,Sommer,Fimmilena,mittelbraun,1499,0.133422,Abteilung005,WHG021,WUG073,0,107.90
1,900001,2014-03-28,2013-10-18,2014-05-31,59.95,59.95,0.0,0.0,2,Sommer,Fimmilena,mittelbraun,1499,0.266845,Abteilung005,WHG021,WUG073,1,119.90
2,900001,2014-03-29,2013-10-18,2014-05-31,59.95,59.95,0.0,0.0,3,Sommer,Fimmilena,mittelbraun,1499,0.466978,Abteilung005,WHG021,WUG073,2,179.85
3,900001,2014-03-31,2013-10-18,2014-05-31,59.95,59.55,0.4,0.0,5,Sommer,Fimmilena,mittelbraun,1499,0.800534,Abteilung005,WHG021,WUG073,4,297.75
4,900001,2014-04-01,2013-10-18,2014-05-31,59.95,59.95,0.0,0.0,6,Sommer,Fimmilena,mittelbraun,1499,1.200801,Abteilung005,WHG021,WUG073,5,359.70


In [4]:
%%time

# expand transaction_date information
season = list()
weekday = list()
months = list()

for i in df.transaction_date:
    month = i.month
    # meteorological seasons
    if 2 < month < 6:
        season.append('spring')
    elif 5 < month < 9:
        season.append("summer")
    elif 8 < month < 12:
        season.append("fall")
    else:
        season.append("winter")

    months.append(month)
    day = i.weekday()
    weekday.append(day)

df['season_buy'] = pd.Series(season, index=df.index)
df['month'] = pd.Series(months, index=df.index)
df['weekday'] = pd.Series(weekday, index=df.index)

CPU times: user 11.1 s, sys: 241 ms, total: 11.4 s
Wall time: 11.4 s


In [5]:
%%time

# expand transaction_date information

def get_season(month):
    if 2 < month < 6:
        return 'spring'
    elif 5 < month < 9:
        return "summer"
    elif 8 < month < 12:
        return "fall"
    else:
        return "winter"

df['month'] = df['transaction_date'].apply(lambda x : x.month)
df['season_buy'] = df['month'].apply(lambda x : get_season(x))
df['weekday'] = df['transaction_date'].apply(lambda x : x.weekday())

CPU times: user 15.9 s, sys: 280 ms, total: 16.2 s
Wall time: 16.1 s
